In [211]:
#default_exp query
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Queries SQL e MongoDB

> Este módulo executa as queries sql / MongoDB necessárias para baixar o Banco do STEL, RADCOM e MOSAICO

In [212]:
#export
import requests
from pathlib import Path
import pandas as pd
import pyodbc
from fastcore.test import * 

In [213]:
#export
ESTACOES = 'http://sistemas.anatel.gov.br/se/public/file/b/srd/estacao_rd.zip'
PLANO_BASICO = 'http://sistemas.anatel.gov.br/se/public/file/b/srd/PlanoBasico.zip'
COL_ESTACOES = ('@SiglaServico', '@state', '@entidade', '@fistel', '@cnpj', '@municipio', '@uf', '@id')
COL_PLANO_BASICO = ('@id', '@Classe', '@Frequencia', '@Latitude', '@Longitude')
STEL = ('Frequência', 'Latitude', 'Longitude', 'Serviço', 'Número da Estação', 'Entidade', 'Município', 'UF')
RADIODIFUSAO = ('Frequência', 'Latitude', 'Longitude', 'Situação', 'Classe', 'Número da Estação', 'Entidade', 'Município', 'UF')
RADCOM = """
       select f.MedFrequenciaInicial as 'Frequência', 
       uf.SiglaUnidadeFrequencia as 'Unidade',
       es.MedLatitudeDecimal as 'Latitude' ,
       es.MedLongitudeDecimal as 'Longitude',
       Sitarweb.dbo.FN_SRD_RetornaIndFase(PB.NumServico, Pr.idtPedidoRadcom) as 'Fase',
       Sitarweb.dbo.FN_SRD_RetornaSiglaSituacao(h.IdtHabilitacao, Es.IdtEstacao) as 'Situação',
       es.NumEstacao as 'Numero da Estação',
       e.NumCnpjCpf as 'CNPJ',
       e.NomeEntidade as 'Entidade',
       m.NomeMunicipio as 'Município',
       pb.SiglaUF as 'UF'
from ENTIDADE e
inner join HABILITACAO h on h.IdtEntidade = e.IdtEntidade
inner join SRD_PEDIDORADCOM pr on pr.IdtHabilitacao = h.IdtHabilitacao
inner join SRD_PLANOBASICO pb on pb.IdtPlanoBasico = pr.IdtPlanoBasico
inner join estacao es on es.IdtHabilitacao = h.IdtHabilitacao
inner join FREQUENCIA f on f.IdtEstacao = es.IdtEstacao
inner join UnidadeFrequencia uf on uf.IdtUnidadeFrequencia = f.IdtUnidadeFrequencia
inner join Municipio m on m.CodMunicipio = pb.CodMunicipio
where h.NumServico = '231'
"""

In [214]:
STEL = """
IF OBJECT_ID('tempDB..##faixas','U') is not null 
drop table ##faixas 
create table ##faixas (id int not null, faixa varchar(20), inic float, fim float,); 
insert into ##faixas values(0,'De 20 MHz - 6 GHz','20000', '6000000'); 

select distinct 
f.MedTransmissaoInicial as 'Frequência',
uf.SiglaUnidadeFrequencia as 'Unidade',
e.MedLatitudeDecimal as 'Latitude',
e.MedLongitudeDecimal as 'Longitude',
e.NumServico as 'Número do Serviço',
e.NumEstacao as 'Número da estação',
ent.NomeEntidade as 'Entidade',
mu.NomeMunicipio as 'Município', 
e.SiglaUf as 'UF',
ent.NumCnpjCpf as 'CNPJ',
h.NumFistel as 'Fistel'
from contrato c
inner join estacao e on e.IdtContrato = c.Idtcontrato
inner join frequencia f on f.IdtEstacao = e.IdtEstacao
inner join HABILITACAO h on h.IdtHabilitacao = c.IdtHabilitacao 
inner join entidade ent on ent.IdtEntidade = h.IdtEntidade 
inner join endereco en on en.IdtEstacao = e.IdtEstacao 
inner join Municipio mu on mu.CodMunicipio = en.CodMunicipio 
inner join Servico s on s.NumServico = h.NumServico and s.IdtServicoAreaAtendimento = 4
left join UnidadeFrequencia uf on uf.IdtUnidadeFrequencia = f.IdtUnidadeTransmissao
left outer join ##faixas fx on (
(fx.inic <= f.MedRecepcaoInicialKHz and fx.fim >= f.MedRecepcaoInicialKHz) 
or (fx.inic <= f.MedTransmissaoInicialKHz and fx.fim >= f.medtransmissaoinicialkhz)
or (fx.inic <= f.MedFrequenciaInicialKHz and fx.fim >= f.MedFrequenciaInicialKHz)
or (fx.inic <= f.MedFrequenciaFinalKHz and fx.fim >= f.MedFrequenciaFinalKHz)
) 
where e.DataExclusao is null and 
fx.faixa is not null and
f.MedTransmissaoInicial is not null
and h.NumServico <> '010'
"""

In [215]:
#export
def connect_db():
    """Conecta ao Banco ANATELBDRO01 e retorna o 'cursor' (iterador) do Banco pronto para fazer iterações"""
    conn = pyodbc.connect(
        "Driver={ODBC Driver 17 for SQL Server};"
        "Server=ANATELBDRO01;"
        "Database=SITARWEB;"
        "Trusted_Connection=yes;"
        "MultipleActiveResultSets=True;"
    )

    return conn

In [216]:
#slow
def test_connection():
    conn = connect_db()
    cursor = conn.cursor()
    for query in (RADCOM, STEL):
        cursor.execute(query)
        test_eq(type(cursor.fetchone()), pyodbc.Row)

In [217]:
#export
def update_radcom():
    """Update the Radcom File querying the Database"""
    conn = connect_db()
    pd.read_sql_query(RADCOM, conn).to_feather('anatelspyder/files/radcom.fth')
    
def update_stel():
    """Update the Stel File querying the Database"""
    conn = connect_db()
    pd.read_sql_query(STEL, conn).to_feather('anatelspyder/files/stel.fth')
    
def update_mosaico():
    """Update the Mosaico File by downloading the zipped xml file from the Spectrum E Web page"""
    file = requests.get(ESTACOES)
    with open('anatelspyder/files/estações.zip', 'wb') as estações:
        estações.write(file.content)
    file = requests.get(PLANO_BASICO)
    with open('anatelspyder/files/plano_basico.zip', 'wb') as plano_basico:
        plano_basico.write(file.content)

In [218]:
from nbdev.export import notebook2script; notebook2script()

Converted index.ipynb.
Converted process.ipynb.
Converted queries.ipynb.
